In [347]:
from pykrx import stock
import pandas as pd

#### 코스피 종목 코드, 이름 추출

In [348]:
kospi_tickers = stock.get_market_ticker_list(market="KOSPI")
# len(kospi_tickers), len(kosdaq_tickers)
df_kospi_ticker = pd.Series(kospi_tickers).to_frame()
df_kospi = df_kospi_ticker.rename(columns={0:'티커'})
df_kospi

,티커
0,095570
1,006840
2,027410
3,282330
4,138930
...,...
949,079980
950,005010
951,000540
952,000545


In [349]:
kospi_names = []
for ticker in kospi_tickers:
    n = stock.get_market_ticker_name(ticker)
    kospi_names.append(n)
df_kospi["종목"] = kospi_names
df_kospi = df_kospi.set_index("티커")
df_kospi

,종목
티커,
095570,AJ네트웍스
006840,AK홀딩스
027410,BGF
282330,BGF리테일
138930,BNK금융지주
...,...
079980,휴비스
005010,휴스틸
000540,흥국화재


#### 주말이면 금요일 날짜로 데이터 가져오기, 공휴일이면 전날로 가져오기

In [350]:
%pip install holidays

Note: you may need to restart the kernel to use updated packages.


In [351]:
"""
holidays 모듈에서 공휴일은 날짜까지만 확인하는 데이터 양식으로 들어간다
"""
import time
from datetime import datetime, date, timedelta
import holidays

today = datetime.today()
kr_holidays = holidays.country_holidays('KR')

while True:
    if today.weekday() == 5:    
        today = today - timedelta(days=1)
    elif today.weekday() == 6:
        today = today - timedelta(days=2)
    elif today.date() in kr_holidays:
        today = today - timedelta(days=1)
    else:
        break

fstr_today = today.strftime("%Y%m%d")
df = stock.get_market_ohlcv(fstr_today, market="KOSPI")
df

,시가,고가,저가,종가,거래량,거래대금,등락률
티커,,,,,,,
095570,4830,4830,4705,4710,77696,367661855,-1.57
006840,16400,16400,16250,16280,3063,49894490,0.00
027410,3950,3970,3880,3920,66821,263379410,-0.76
282330,132700,133800,131900,132100,44355,5882332000,-1.34
138930,7500,7600,7440,7500,2356768,17707496700,0.00
...,...,...,...,...,...,...,...
079980,3945,4080,3920,4030,124545,500267610,1.77
005010,4980,4985,4900,4915,298713,1472128235,-1.31
000540,4805,5050,4805,4905,228540,1122550700,1.87


In [352]:
df_kospi = df_kospi.join(df)
df_kospi

,종목,시가,고가,저가,종가,거래량,거래대금,등락률
티커,,,,,,,,
095570,AJ네트웍스,4830,4830,4705,4710,77696,367661855,-1.57
006840,AK홀딩스,16400,16400,16250,16280,3063,49894490,0.00
027410,BGF,3950,3970,3880,3920,66821,263379410,-0.76
282330,BGF리테일,132700,133800,131900,132100,44355,5882332000,-1.34
138930,BNK금융지주,7500,7600,7440,7500,2356768,17707496700,0.00
...,...,...,...,...,...,...,...,...
079980,휴비스,3945,4080,3920,4030,124545,500267610,1.77
005010,휴스틸,4980,4985,4900,4915,298713,1472128235,-1.31
000540,흥국화재,4805,5050,4805,4905,228540,1122550700,1.87


#### 코스피 펀더멘탈 지수 BPS, PER, PBR, EPS, DIV, DPS

In [369]:
df_kospi_fund = stock.get_market_fundamental(fstr_today, market="KOSPI")
df_kospi_fund.info

<bound method DataFrame.info of           BPS    PER   PBR    EPS   DIV   DPS
티커                                           
095570   8076  23.43  0.58    201  5.73   270
006840  41948   0.00  0.39      0  1.23   200
027410  16528  15.87  0.24    247  2.81   110
282330  55724  11.79  2.37  11203  3.10  4100
138930  30468   3.12  0.25   2404  8.33   625
...       ...    ...   ...    ...   ...   ...
079980  11585   0.00  0.35      0  0.00     0
005010  16181   0.87  0.30   5675  7.12   350
000540  11208   2.29  0.44   2142  0.00     0
000545      0   0.00  0.00      0  0.00     0
003280    544  29.41  5.08     94  0.00     0

[925 rows x 6 columns]>

#### df_kospi 와 df_kospi_fund 인덱스 차이 비교

In [370]:
diff_idx = df_kospi.index.difference(df_kospi_fund.index)
diff_idx

Index(['088260', '088980', '094800', '140910', '145270', '152550', '168490',
       '204210', '293940', '330590', '334890', '338100', '348950', '350520',
       '357120', '357250', '357430', '365550', '377190', '395400', '396690',
       '400760', '404990', '417310', '432320', '448730', '451800', '900140',
       '950210'],
      dtype='object', name='티커')

In [371]:
df_kospi_tmp = df_kospi.merge(df_kospi_fund, how="inner", on="티커")
df_kospi_tmp

,종목,시가,고가,저가,종가,거래량,거래대금,등락률,BPS,PER,PBR,EPS,DIV,DPS
티커,,,,,,,,,,,,,,
095570,AJ네트웍스,4830,4830,4705,4710,77696,367661855,-1.57,8076,23.43,0.58,201,5.73,270
006840,AK홀딩스,16400,16400,16250,16280,3063,49894490,0.00,41948,0.00,0.39,0,1.23,200
027410,BGF,3950,3970,3880,3920,66821,263379410,-0.76,16528,15.87,0.24,247,2.81,110
282330,BGF리테일,132700,133800,131900,132100,44355,5882332000,-1.34,55724,11.79,2.37,11203,3.10,4100
138930,BNK금융지주,7500,7600,7440,7500,2356768,17707496700,0.00,30468,3.12,0.25,2404,8.33,625
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
079980,휴비스,3945,4080,3920,4030,124545,500267610,1.77,11585,0.00,0.35,0,0.00,0
005010,휴스틸,4980,4985,4900,4915,298713,1472128235,-1.31,16181,0.87,0.30,5675,7.12,350
000540,흥국화재,4805,5050,4805,4905,228540,1122550700,1.87,11208,2.29,0.44,2142,0.00,0


#### 지표에 값이 0이거나 Nan 인 종목 제거

In [372]:
df_kospi_clear = df_kospi_tmp[(df_kospi_tmp.BPS != 0) & (df_kospi_tmp.EPS != 0)]
df_kospi_clear

,종목,시가,고가,저가,종가,거래량,거래대금,등락률,BPS,PER,PBR,EPS,DIV,DPS
티커,,,,,,,,,,,,,,
095570,AJ네트웍스,4830,4830,4705,4710,77696,367661855,-1.57,8076,23.43,0.58,201,5.73,270
027410,BGF,3950,3970,3880,3920,66821,263379410,-0.76,16528,15.87,0.24,247,2.81,110
282330,BGF리테일,132700,133800,131900,132100,44355,5882332000,-1.34,55724,11.79,2.37,11203,3.10,4100
138930,BNK금융지주,7500,7600,7440,7500,2356768,17707496700,0.00,30468,3.12,0.25,2404,8.33,625
001460,BYC,422000,434000,421500,429500,1876,801196500,2.14,825735,10.16,0.52,42272,0.70,3000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
081660,휠라홀딩스,39450,39900,38400,38400,142043,5523299750,-3.64,31226,6.71,1.23,5720,4.11,1580
005870,휴니드,6780,6970,6780,6920,240166,1658438910,2.22,11026,9.79,0.63,707,0.00,0
005010,휴스틸,4980,4985,4900,4915,298713,1472128235,-1.31,16181,0.87,0.30,5675,7.12,350


#### ROE 컬럼 추가 (최종 df)

In [373]:
df = df_kospi_clear.copy()
df["ROE"] = (df["EPS"] / df["BPS"]) * 100
df

,종목,시가,고가,저가,종가,거래량,거래대금,등락률,BPS,PER,PBR,EPS,DIV,DPS,ROE
티커,,,,,,,,,,,,,,,
095570,AJ네트웍스,4830,4830,4705,4710,77696,367661855,-1.57,8076,23.43,0.58,201,5.73,270,2.488856
027410,BGF,3950,3970,3880,3920,66821,263379410,-0.76,16528,15.87,0.24,247,2.81,110,1.494434
282330,BGF리테일,132700,133800,131900,132100,44355,5882332000,-1.34,55724,11.79,2.37,11203,3.10,4100,20.104443
138930,BNK금융지주,7500,7600,7440,7500,2356768,17707496700,0.00,30468,3.12,0.25,2404,8.33,625,7.890246
001460,BYC,422000,434000,421500,429500,1876,801196500,2.14,825735,10.16,0.52,42272,0.70,3000,5.119318
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
081660,휠라홀딩스,39450,39900,38400,38400,142043,5523299750,-3.64,31226,6.71,1.23,5720,4.11,1580,18.318068
005870,휴니드,6780,6970,6780,6920,240166,1658438910,2.22,11026,9.79,0.63,707,0.00,0,6.412117
005010,휴스틸,4980,4985,4900,4915,298713,1472128235,-1.31,16181,0.87,0.30,5675,7.12,350,35.071998


#### PBR < 1, ROE > 15, PER < 1 기업

In [374]:
df1 = df.copy()
df1 = df1[(df1["PBR"] < 1) & (df1["ROE"] > 15) & (df1["PER"] < 1)]
df1.sort_values(by="ROE", ascending=False)

,종목,시가,고가,저가,종가,거래량,거래대금,등락률,BPS,PER,PBR,EPS,DIV,DPS,ROE
티커,,,,,,,,,,,,,,,
011200,HMM,18310,18630,18260,18270,1775348,32596466380,-0.54,42300,0.89,0.43,20556,6.57,1200,48.595745
004870,티웨이홀딩스,494,496,488,494,132292,64920966,-0.20,1505,0.71,0.33,697,0.00,0,46.312292
005010,휴스틸,4980,4985,4900,4915,298713,1472128235,-1.31,16181,0.87,0.30,5675,7.12,350,35.071998
000950,전방,22700,23500,22400,22600,1110,25097400,-0.44,184874,0.38,0.12,59197,4.42,1000,32.020187


#### 코스닥 종목 추출

In [375]:
kosdaq_tickers = stock.get_market_ticker_list(market="KOSDAQ")
df_kosdaq_ticker = pd.Series(kosdaq_tickers).to_frame()
df_kosdaq = df_kosdaq_ticker.rename(columns={0:'티커'})
df_kosdaq

,티커
0,060310
1,054620
2,265520
3,211270
4,139050
...,...
1709,024060
1710,010240
1711,189980
1712,037440


In [376]:
kosdaq_names = []
for ticker in kosdaq_tickers:
    n = stock.get_market_ticker_name(ticker)
    kosdaq_names.append(n)
df_kosdaq["종목"] = kosdaq_names
df_kosdaq = df_kosdaq.set_index("티커")
df_kosdaq

,종목
티커,
060310,3S
054620,APS
265520,AP시스템
211270,AP위성
139050,BF랩스
...,...
024060,흥구석유
010240,흥국
189980,흥국에프엔비


In [378]:
import time
from datetime import datetime, date, timedelta

now = datetime.now()
today = datetime.today()
kr_holidays = holidays.country_holidays('KR')

while True:
    if today.weekday() == 5:    
        today = today - timedelta(days=1)
    elif today.weekday() == 6:
        today = today - timedelta(days=2)
    elif today.date() in kr_holidays:
        today = today - timedelta(days=1)
    else:
        break

fstr_today = today.strftime("%Y%m%d")
df_kq = stock.get_market_ohlcv(fstr_today, market="KOSDAQ")
df_kq

,시가,고가,저가,종가,거래량,거래대금,등락률
티커,,,,,,,
060310,2890,2905,2805,2845,910834,2587599380,-2.23
054620,6860,7060,6780,6900,32362,224951120,-0.58
265520,20950,21000,20650,20850,27755,577659950,-0.48
211270,15660,15660,15280,15370,111731,1718494140,-1.85
139050,7810,7810,7230,7560,143226,996697600,0.13
...,...,...,...,...,...,...,...
024060,9970,10320,9950,10010,540572,5461603820,-0.50
010240,5740,5780,5740,5780,17651,101769020,0.17
189980,2200,2200,2125,2125,186988,404266765,-3.41


In [379]:
df_kq = df_kosdaq.join(df_kq)
df_kq

,종목,시가,고가,저가,종가,거래량,거래대금,등락률
티커,,,,,,,,
060310,3S,2890,2905,2805,2845,910834,2587599380,-2.23
054620,APS,6860,7060,6780,6900,32362,224951120,-0.58
265520,AP시스템,20950,21000,20650,20850,27755,577659950,-0.48
211270,AP위성,15660,15660,15280,15370,111731,1718494140,-1.85
139050,BF랩스,7810,7810,7230,7560,143226,996697600,0.13
...,...,...,...,...,...,...,...,...
024060,흥구석유,9970,10320,9950,10010,540572,5461603820,-0.50
010240,흥국,5740,5780,5740,5780,17651,101769020,0.17
189980,흥국에프엔비,2200,2200,2125,2125,186988,404266765,-3.41


#### 코스닥 펀더멘탈 지수 BPS, PER, PBR, EPS, DIV, DPS

In [380]:
df_kosdaq_fund = stock.get_market_fundamental(fstr_today, market="KOSDAQ")
df_kosdaq_fund.info

<bound method DataFrame.info of           BPS    PER   PBR   EPS   DIV  DPS
티커                                         
060310    947  94.83  3.00    30  0.00    0
054620  10864  13.66  0.64   505  0.00    0
265520  17980   3.82  1.16  5463  1.29  270
211270   5997  53.55  2.56   287  0.46   70
139050   5010   0.00  1.51     0  0.00    0
...       ...    ...   ...   ...   ...  ...
024060   5508  54.70  1.82   183  1.50  150
010240   7971   7.81  0.73   740  3.81  220
189980   2295   6.88  0.93   309  1.88   40
037440   5186  12.31  1.35   567  2.15  150
238490   5656   0.00  1.09     0  0.00    0

[1694 rows x 6 columns]>

In [381]:
diff_idx = df_kosdaq.index.difference(df_kosdaq_fund.index)
diff_idx

Index(['900070', '900100', '900110', '900120', '900250', '900260', '900270',
       '900280', '900290', '900300', '900310', '900340', '950110', '950130',
       '950140', '950160', '950170', '950190', '950200', '950220'],
      dtype='object', name='티커')

In [382]:
df_kosdaq_tmp = df_kq.merge(df_kosdaq_fund, how="inner", on="티커")
df_kosdaq_tmp

,종목,시가,고가,저가,종가,거래량,거래대금,등락률,BPS,PER,PBR,EPS,DIV,DPS
티커,,,,,,,,,,,,,,
060310,3S,2890,2905,2805,2845,910834,2587599380,-2.23,947,94.83,3.00,30,0.00,0
054620,APS,6860,7060,6780,6900,32362,224951120,-0.58,10864,13.66,0.64,505,0.00,0
265520,AP시스템,20950,21000,20650,20850,27755,577659950,-0.48,17980,3.82,1.16,5463,1.29,270
211270,AP위성,15660,15660,15280,15370,111731,1718494140,-1.85,5997,53.55,2.56,287,0.46,70
139050,BF랩스,7810,7810,7230,7560,143226,996697600,0.13,5010,0.00,1.51,0,0.00,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
024060,흥구석유,9970,10320,9950,10010,540572,5461603820,-0.50,5508,54.70,1.82,183,1.50,150
010240,흥국,5740,5780,5740,5780,17651,101769020,0.17,7971,7.81,0.73,740,3.81,220
189980,흥국에프엔비,2200,2200,2125,2125,186988,404266765,-3.41,2295,6.88,0.93,309,1.88,40


In [383]:
df_kosdaq_clear = df_kosdaq_tmp[(df_kosdaq_tmp.BPS != 0) & (df_kosdaq_tmp.EPS != 0)]
df_kosdaq_clear

,종목,시가,고가,저가,종가,거래량,거래대금,등락률,BPS,PER,PBR,EPS,DIV,DPS
티커,,,,,,,,,,,,,,
060310,3S,2890,2905,2805,2845,910834,2587599380,-2.23,947,94.83,3.00,30,0.00,0
054620,APS,6860,7060,6780,6900,32362,224951120,-0.58,10864,13.66,0.64,505,0.00,0
265520,AP시스템,20950,21000,20650,20850,27755,577659950,-0.48,17980,3.82,1.16,5463,1.29,270
211270,AP위성,15660,15660,15280,15370,111731,1718494140,-1.85,5997,53.55,2.56,287,0.46,70
126600,BGF에코머티리얼즈,4520,4560,4490,4490,115381,520746015,-1.64,6585,4.26,0.68,1053,2.23,100
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
145020,휴젤,184100,187000,181200,183900,78010,14376024500,-1.08,67095,38.22,2.74,4811,0.00,0
024060,흥구석유,9970,10320,9950,10010,540572,5461603820,-0.50,5508,54.70,1.82,183,1.50,150
010240,흥국,5740,5780,5740,5780,17651,101769020,0.17,7971,7.81,0.73,740,3.81,220


In [384]:
df_kq = df_kosdaq_clear.copy()
df_kq["ROE"] = (df_kq["EPS"] / df_kq["BPS"]) * 100
df_kq

,종목,시가,고가,저가,종가,거래량,거래대금,등락률,BPS,PER,PBR,EPS,DIV,DPS,ROE
티커,,,,,,,,,,,,,,,
060310,3S,2890,2905,2805,2845,910834,2587599380,-2.23,947,94.83,3.00,30,0.00,0,3.167899
054620,APS,6860,7060,6780,6900,32362,224951120,-0.58,10864,13.66,0.64,505,0.00,0,4.648380
265520,AP시스템,20950,21000,20650,20850,27755,577659950,-0.48,17980,3.82,1.16,5463,1.29,270,30.383760
211270,AP위성,15660,15660,15280,15370,111731,1718494140,-1.85,5997,53.55,2.56,287,0.46,70,4.785726
126600,BGF에코머티리얼즈,4520,4560,4490,4490,115381,520746015,-1.64,6585,4.26,0.68,1053,2.23,100,15.990888
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
145020,휴젤,184100,187000,181200,183900,78010,14376024500,-1.08,67095,38.22,2.74,4811,0.00,0,7.170430
024060,흥구석유,9970,10320,9950,10010,540572,5461603820,-0.50,5508,54.70,1.82,183,1.50,150,3.322440
010240,흥국,5740,5780,5740,5780,17651,101769020,0.17,7971,7.81,0.73,740,3.81,220,9.283653


In [385]:
df_kq = df_kq.copy()
df_kq = df_kq[(df_kq["PBR"] < 1) & (df_kq["ROE"] > 15) & (df_kq["PER"] < 1)]
df_kq.sort_values(by="ROE", ascending=False)

,종목,시가,고가,저가,종가,거래량,거래대금,등락률,BPS,PER,PBR,EPS,DIV,DPS,ROE
티커,,,,,,,,,,,,,,,
106240,파인테크닉스,1580,1580,1556,1563,52785,82713601,-1.08,5583,0.22,0.28,7096,0.00,0,127.100125
151860,KG ETS,10480,11520,10390,10760,1994240,21948201420,2.67,22772,0.79,0.47,13572,1.39,150,59.599508
035080,그래디언트,13320,15890,13260,13870,1256260,18319733010,3.58,40293,0.65,0.34,21470,1.44,200,53.284690
039560,다산네트웍스,4005,4035,3915,3985,190700,755214815,-0.50,10652,0.72,0.37,5509,0.00,0,51.717987
140520,대창스틸,2555,2565,2525,2555,8630,21931135,-0.58,7269,0.97,0.35,2641,5.87,150,36.332370
104480,티케이케미칼,1824,1824,1805,1807,104794,189705606,-0.06,10429,0.60,0.17,3028,0.00,0,29.034423
